# 상점 신용카드 매출 예측

In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from tqdm import tqdm

## 데이터 불러오기

In [11]:
raw_data=pd.read_csv('./funda_train.csv')
data=raw_data.copy()
data.info()
#data.describe()  
# 날짜 헷갈리니까 합친다.
# 환불 금액이 보임. 환불을 먼저 없앤다.

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

## 환불금액제거.

In [3]:
data

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
0,0,0,b,2016-06-01,13:13,0,NaN,기타 미용업,1857.142857
1,0,1,h,2016-06-01,18:12,0,NaN,기타 미용업,857.142857
2,0,2,c,2016-06-01,18:52,0,NaN,기타 미용업,2000.000000
3,0,3,a,2016-06-01,20:22,0,NaN,기타 미용업,7857.142857
4,0,4,c,2016-06-02,11:06,0,NaN,기타 미용업,2000.000000
...,...,...,...,...,...,...,...,...,...
6556608,2136,4663855,d,2019-02-28,23:20,0,제주 제주시,기타 주점업,-4500.000000
6556609,2136,4663855,d,2019-02-28,23:24,0,제주 제주시,기타 주점업,4142.857143
6556610,2136,4663489,a,2019-02-28,23:24,0,제주 제주시,기타 주점업,4500.000000
6556611,2136,4663856,d,2019-02-28,23:27,0,제주 제주시,기타 주점업,571.428571


In [4]:
data_neg=data[data['amount']<0]
data_neg

,store_id,card_id,card_company,transacted_date,transacted_time,installment_term,region,type_of_business,amount
41,0,40,a,2016-06-10,17:26,2,NaN,기타 미용업,-8571.428571
347,0,285,a,2016-08-04,17:52,0,NaN,기타 미용업,-1857.142857
731,0,473,g,2016-10-17,10:32,0,NaN,기타 미용업,-2000.000000
831,0,230,b,2016-11-03,15:36,0,NaN,기타 미용업,-85.714286
944,0,138,a,2016-11-28,13:21,0,NaN,기타 미용업,-57.142857
...,...,...,...,...,...,...,...,...,...
6556242,2136,4663626,b,2019-02-01,21:19,0,제주 제주시,기타 주점업,-13428.571429
6556448,2136,4663760,d,2019-02-15,00:46,0,제주 제주시,기타 주점업,-6928.571429
6556485,2136,4663779,b,2019-02-18,02:45,0,제주 제주시,기타 주점업,-5571.428571
6556489,2136,4663780,d,2019-02-18,21:43,0,제주 제주시,기타 주점업,-8571.428571


In [5]:
#합치기
data['date']=data['transacted_date']+' '+data['transacted_time']
data['date']=pd.to_datetime(data['date']) # str형태를 시간형태로 바꿔 줘야 함 
#환불 제거
data_neg=data[data['amount']<0]
data_pos=data[data['amount']>0]
remove_data=pd.DataFrame()
# store_id 안나누고 하니까 시간이 너무 오래걸림. 
for i in tqdm(data.store_id.unique()):
    divided_data=data_pos[data_pos['store_id']==i] 
    divided_data2=data_neg[data_neg['store_id']==i] 
    #행 마다 조사.    
    for neg in divided_data2.to_records():  
        refund_store=neg['store_id']
        refund_id=neg['card_id'] 
        refund_date=neg['date']
        refund_amount=abs(neg['amount'])
        #행뽑기
        row=divided_data[divided_data['date']<=refund_date]
        row=row[row['card_id']==refund_id]
        row=row[row['amount']==refund_amount]
        
        if(len(row)!=0):
            refund_date=max(row['date']) 
            remove=divided_data[divided_data['date']==refund_date] 
            remove=remove[remove['card_id']==refund_id] 
            remove=remove[remove['amount']==refund_amount] 
            divided_data=divided_data.drop(index=remove.index) 

    remove_data=pd.concat([remove_data,divided_data],axis=0)

remove_data

#remove_data.to_csv('./funda_remove_data.csv')

KeyboardInterrupt: 

In [5]:
remove_data.to_csv('./funda_remove_data.csv')

In [ ]:
data['transacted_date'] = pd.to_datetime(data['transacted_date'])
data = data.set_index('transacted_date')

In [6]:
data=pd.read_csv('./funda_remove_data.csv',index_col=0)
data['date_slice']=pd.to_datetime(data['date'].str.slice(stop=10))
data=data.set_index('date_slice')
#data.head(3)

D:\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:583: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [7]:
data_month=pd.DataFrame()
for i in tqdm(data.store_id.unique()):
    data_num=data[data.store_id==i]
    sum_amount=data_num['amount'].resample(rule='m').sum()
    data_mon=pd.concat([sum_amount],axis=1)
    data_mon.insert(0,'store_id',i)
    data_month=pd.concat([data_month,data_mon],axis=0)
    
#data_month.head(10)

100%|███████████████████████████████████████████████████████████| 1967/1967 [00:23<00:00, 83.84it/s]


## ARIMA

In [12]:
import itertools

p = [0,1,2]
d = [0,1,2]
q = [0,1,2]

pdq = list(itertools.product(p, d, q))

In [13]:
from statsmodels.tsa.arima.model import ARIMA

arima_pred_arr = np.array([])
for i in tqdm(data_month.store_id.unique()):
    data_set = data_month[data_month.store_id == i]

    best_score = 10000000000
    best_param = 0
        
    for param in pdq:
        try:
            arima_model = ARIMA(data_set.amount.values, order=(param[0], param[1], param[2]))
            result = arima_model.fit()
            if result.aic < best_score:
                best_score = result.aic
                best_param = param

        except:
            continue

    arima_model = ARIMA(data_set.amount.values, order=best_param)
    arima_result = arima_model.fit()
    arima_pred = arima_result.forecast(3)[0]

    arima_pred_arr = np.concatenate((arima_pred_arr, np.array([arima_pred.sum()])))

NameError: name 'data_month' is not defined

In [ ]:
arima_sum_sub = pd.DataFrame({'store_id' : data_month.store_id.unique().tolist(), 'amount' : arima_pred_arr})
arima_sum_sub.to_csv('./funda_arima_sub.csv', index=False)

In [ ]:
arima_sum_sub